In [9]:
import numpy as np
import pandas as pd
import os
import json

import torch

In [11]:
#Combine json files
dataDir = '../WebScraper/data/'
json_files = [os.path.join(dataDir, f) for f in os.listdir(dataDir) if f.endswith('.json')]
objects = []
for file in json_files:
    with open(file, 'r') as f:
        objects.append(json.load(f))
with open('data.json', 'w') as f:
    json.dump(objects, f, indent=4)